### Install required packages

In [ ]:
%pip install llama-index llama-index-llms-ollama

### Chat with llm

In [ ]:
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings

llm = Ollama(model="llama3.1", request_timeout=120.0)

Settings.llm = llm

In [66]:
from llama_index.core.llms import ChatMessage, MessageRole

schema = ""

# read sql file
with open("./data/emp.sql") as f:
    schema = f.read()

prompt_template = """
    You are a senior business analyst at a large company.
    You can to analyse the relationship between different tables in a sql database(mysql here)

    ## Skills
    1. Summary of the relationships between different tables in the database.
    2. Ability to generate sql queries to extract data from the database.
    3. Ability to understand the user requirements and provide the correct solution.
    4. Ability to reason on the correctness of the sql query generated.
    5. You do not assume the user intent and ask for clarification when needed.
    
    The Schema for the database is as follows:

    -------

    {schema}

    ------

    ## Instructions:
    1. Your answer should be concise and to the point.
    2. If you have difficulty understanding the user requirements ask for clarification.
    3. Don't add or assument additional database schema or data that is not provided.
    4. Never assume anything regarding the user query that is not provided in the prompt.
    5. When there are multiple ways to understand the user query,list the possible queries that user needs to clarify before writing query.
    6. You need to prioritise clarification on the user requirements before generating all possible queries.

    When you are confused or have multiple ways in understanding the user query:
    ## Output:
        - Clarification: List of questions that you would ask the user to clarify the requirements.

    When asked to generate a query, provide the following:
    ## Output:
        - Summary: summary of user requirements from the query.
        - Query: sql query that you would use to extract the data from the database.
        - Alternate Queries: If you have written multiple queries, provide them here.
        - Explaination: Provide a brief explanation of the query you have written.

    When asked to provide a summary, provide the following:
    ## Output:
        - Summary: summary of the relationships between different tables in the database.
"""

# append schema to the prompt
prompt_template = prompt_template.format(schema=schema)

messages =[
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=prompt_template,
    )
]

print(messages)

[ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content="\n    You are a senior business analyst at a large company.\n    You can to analyse the relationship between different tables in a sql database(mysql here)\n\n    ## Skills\n    1. Summary of the relationships between different tables in the database.\n    2. Ability to generate sql queries to extract data from the database.\n    3. Ability to understand the user requirements and provide the correct solution.\n    4. Ability to reason on the correctness of the sql query generated.\n    5. You do not assume the user intent and ask for clarification when needed.\n    \n    The Schema for the database is as follows:\n\n    -------\n\n    \n    CREATE TABLE employees (\n        emp_no      INT             NOT NULL,\n        birth_date  DATE            NOT NULL,\n        first_name  VARCHAR(14)     NOT NULL,\n        last_name   VARCHAR(16)     NOT NULL,\n        gender      ENUM ('M','F')  NOT NULL,    \n        hire_date   DATE   

In [67]:
query = "fetch the salaries of each department"

messages.append(ChatMessage(role=MessageRole.USER, content=query))

response = llm.chat(messages)

print(response)
messages.append(ChatMessage(role=response.message.role, content=response.message.content))

assistant: ## Clarification
- What do you mean by "salaries of each department"? Are you looking for the average salary per department or just the total sum of salaries within each department?
- Do you want to consider only the current employees (i.e., those who are still in a department) or all employees who have ever been in a department?

## Summary
You want to fetch the salaries for each department, but I need clarification on whether it's an average salary per department or total sum of salaries.

## Output 1: Assuming Average Salary per Department
If we assume you want the average salary per department, here's one possible query:

```sql
SELECT d.dept_name, AVG(s.salary) as avg_salary
FROM departments d 
JOIN dept_emp de ON d.dept_no = de.dept_no
JOIN salaries s ON de.emp_no = s.emp_no
GROUP BY d.dept_name;
```

Explanation: This query joins the `departments`, `dept_emp`, and `salaries` tables on their respective keys. It groups the results by department name and calculates the a

In [64]:
query = "fetch the name of the employees who are working in the department with the highest salary"

messages.append(ChatMessage(role=MessageRole.USER, content=query))

response = llm.chat(messages)

print(response)

messages.append(ChatMessage(role=response.message.role, content=response.message.content))

assistant: ## Clarification
- What does it mean by "highest salary"?
  * Is it the highest average salary for each department?
  * Or is it the highest total salary paid to employees in a particular department?

## Output


Assuming the first interpretation (average salary per department), here's my response:


## Summary

* Find the department with the highest average salary.
* Get the names of employees working in that department.

## Query
```sql
WITH 
  avg_salaries AS (
    SELECT 
      d.dept_name,
      AVG(s.salary) as avg_salary
    FROM 
      salaries s
    JOIN 
      dept_emp de ON s.emp_no = de.emp_no
    JOIN 
      departments d ON de.dept_no = d.dept_no
    GROUP BY 
      d.dept_name
  )
SELECT 
  e.first_name,
  e.last_name
FROM 
  avg_salaries AS a
JOIN 
  dept_emp de ON a.dept_name = de.dept_no
JOIN 
  employees e ON de.emp_no = e.emp_no
WHERE 
  a.avg_salary = (SELECT MAX(avg_salary) FROM avg_salaries);
```

## Alternate Queries

If the second interpretation is c

In [65]:
query = "how many number of employees are working in each department"

messages.append(ChatMessage(role=MessageRole.USER, content=query))

response = llm.chat(messages)

print(response)

messages.append(ChatMessage(role=response.message.role, content=response.message.content))

assistant: ## Clarification
- What does it mean by "number of employees"?
  * Does it refer to the count of all employees, including past and present ones?
  * Or is it referring to the count of active employees (i.e., those who have not left the company)?

## Output


Assuming the first interpretation (count of all employees), here's my response:


## Summary

* Count the total number of employees in each department.

## Query
```sql
SELECT 
    d.dept_name,
    COUNT(de.emp_no) as num_employees
FROM 
    dept_emp de
JOIN 
    departments d ON de.dept_no = d.dept_no
GROUP BY 
    d.dept_name;
```

## Alternate Queries

If the second interpretation is correct (count of active employees), here's an alternative query:

```sql
SELECT 
    d.dept_name,
    COUNT(de.emp_no) as num_employees
FROM 
    dept_emp de
JOIN 
    departments d ON de.dept_no = d.dept_no
WHERE 
    de.to_date > CURRENT_DATE
GROUP BY 
    d.dept_name;
```

## Explanation

In the first query, we count the total number 